In [746]:
from gurobipy import *
import numpy as np
import pandas as pd
from Hourbid import *
from Fibid import *
from blockbid import *


In [747]:
HourBid()[1]

,ID,OrderID,Hour,BitType,Quantity,price,Duration,LinkID
0,12,1000,1,S,160800.0,2000.00,1,NaN
1,12,999,1,S,160832.0,1996.67,1,NaN
2,12,998,1,S,160844.0,1995.44,1,NaN
3,12,997,1,S,160870.0,1992.77,1,NaN
4,12,996,1,S,160886.0,1991.23,1,NaN
...,...,...,...,...,...,...,...,...
23995,242,5,24,S,197793.0,6.02,1,NaN
23996,242,4,24,S,197864.0,5.64,1,NaN
23997,242,3,24,S,198583.0,2.48,1,NaN
23998,242,2,24,S,199180.0,0.83,1,NaN


In [748]:
BS_bids = Blockbid()[0]
BD_bids = Blockbid()[1]
HS_bids = HourBid()[0]
HD_bids = HourBid()[1]
FS_bids = Fibid()[0]
FD_bids = Fibid()[1]

In [749]:
np.shape(BD_bids)

(85, 8)

In [750]:
L_D = len(BD_bids)
L_S = len(Blockbid()[0])

L_S_link_ID = np.zeros(L_S,dtype = int)

for i in range(L_S):

    if BS_bids.isnull()['LinkID'][i] == True:
        L_S_link_ID[i] = int(1)
    elif BS_bids.isnull()['LinkID'][i] == False:
         L_S_link_ID[i] = int(0)


L_D_link_ID = np.zeros(L_D,dtype=int)

for i in range(L_D):

    if True == BD_bids.isnull()['LinkID'][i]:

        L_D_link_ID[i] = int(1)
    elif False == BD_bids.isnull()['LinkID'][i]:
         L_D_link_ID[i] = int(0)



print(L_S_link_ID)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1]


In [751]:
Delta_S = np.zeros([L_S,24])


Delta_D = np.zeros([L_D,24])
BD_bids  = Blockbid()[1]
for i in range (L_D):
    if BD_bids['Hour'][i]+BD_bids['Duration'][i]<=25:
        Delta_D[i,BD_bids['Hour'][i]-1:(BD_bids['Hour'][i]+BD_bids['Duration'][i])-1] = 1

BS_bids = Blockbid()[0]
for i in range (L_S):
    if BS_bids['Hour'][i]+BS_bids['Duration'][i]<=25:
        Delta_S[i,BS_bids['Hour'][i]-1:(BS_bids['Hour'][i]+BS_bids['Duration'][i])-1] = 1

In [752]:
def find_F(Del,leng,bids):
    Res = [[] for i in range(24)]
    Res_max = [[] for i in range(24)]
    Res_min = [[] for i in range(24)]
    for i in range (24):
        for j in range(leng):
            if Del[j,i] == 1:
                Res[i].append(bids['price'][j])
    for i in range(24):
        Res_max[i] = np.max(Res[i])
        Res_min[i] = np.min(Res[i])
    return(Res_max,Res_min)

FD_min = np.min(FD_bids['price'])
FD_max = np.max(FD_bids['price'])
FS_min = np.min(FS_bids['price'])
FS_max = np.max(FS_bids['price'])

In [753]:
HD_bids['price'][HIndex_D[1][1]]

0.0

In [754]:
HIndex_D[1][1]

1999

In [755]:
DMax_Res = [[] for i in range(24)]
Dm_Res = [[] for i in range(24)]
SMax_Res = [[] for i in range(24)]
Sm_Res = [[] for i in range(24)]

for i in range(24):
    DMax_Res[i].append(HD_bids['price'][HIndex_D[i][0]])
    DMax_Res[i].append(find_F(Delta_D,L_D,BD_bids)[0][i])
    DMax_Res[i].append(FD_max)
    
for i in range(24):
    SMax_Res[i].append(HS_bids['price'][HIndex_S[i][1]])
    SMax_Res[i].append(find_F(Delta_S,L_S,BS_bids)[0][i])
    SMax_Res[i].append(FS_max)
    
for i in range(24):
    Dm_Res[i].append(HD_bids['price'][HIndex_D[i][1]])
    Dm_Res[i].append(find_F(Delta_D,L_D,BD_bids)[1][i])
    Dm_Res[i].append(FD_min)
    
for i in range(24):
    Sm_Res[i].append(HS_bids['price'][HIndex_S[i][0]])
    Sm_Res[i].append(find_F(Delta_S,L_S,BS_bids)[1][i])
    Sm_Res[i].append(FS_min)

In [756]:
def findmin(num1,num2,num3):
    
    return(min([num1,num2,num3]))

def findmax(num1,num2,num3):

    return(max([num1,num2,num3]))


def put_into_list(maximums,minimums):
    F_min = []
    F_max = []
    for x in range(24):
        F_min.append(findmin(minimums[x][0],minimums[x][1],minimums[x][2]))
        F_max.append(findmax(maximums[x][0],maximums[x][1],maximums[x][2]))

    return(F_min,F_max)

In [757]:
E = Model("Energy")
E.params.NonConvex =2
E.update()
J = np.arange(1,find_largest(HIndex_D)+1)
K = np.arange(1,find_largest(HIndex_S)+1)
H = np.arange(1,25)

Set parameter NonConvex to value 2


In [758]:
L_FD = len(Fibid()[1])
L_FS = len(Fibid()[0])
L_FS

29

In [759]:
xD = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS, name='xD')
xS = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS, name='xS')
w_D = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.BINARY,name ='W_D')
w_S = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.BINARY,name ='W_S')
yD = E.addVars(L_D,ub=1,lb=0,  vtype=GRB.BINARY,name ='yD')
yS = E.addVars(L_S,ub=1,lb=0,  vtype=GRB.BINARY,name ='yS')
F_z = E.addVars(H,lb = 0,vtype=GRB.CONTINUOUS, name='F_Z')
# F_zD = E.addVars(H,lb = 0,vtype=GRB.CONTINUOUS, name='F_ZD')
V_S = E.addVars(H,L_FS,lb=0,vtype=GRB.BINARY,name = 'V_S')
V_D = E.addVars(H,L_FD,lb=0,vtype=GRB.BINARY,name = 'V_D')

In [760]:
E.setObjective(sum(0.5*(2*Pair(j,h,HD_bids,'DP')[0]+xD[h,j]*(Pair(j,h,HD_bids,'DP')[1]-Pair(j,h,HD_bids,'DP')[0]))*xD[h,j]*(Pair(j,h,HD_bids,'DQ')[1]-Pair(j,h,HD_bids,'DQ')[0])for j in J for h in H)
                -sum(0.5*(2*Pair(j,h,HS_bids,'SP')[0]+xS[h,j]*(Pair(j,h,HS_bids,'SP')[1]-Pair(j,h,HS_bids,'SP')[0]))*xS[h,j]*(Pair(j,h,HS_bids,'SQ')[1]-Pair(j,h,HS_bids,'SQ')[0])for j in J for h in H)
                +sum(BD_bids['Duration'][i]*BD_bids['Quantity'][i]*BD_bids['price'][i]*yD[i] for i in range(L_D)) - sum(BS_bids['Duration'][i]*BS_bids['Quantity'][i]*BS_bids['price'][i]*yS[i] for i in range(L_S))
                +sum(FD_bids['Quantity'][i]*FD_bids['price'][i] for i in range(L_FD))*sum(V_D[h,i]for i in range(L_FD) for h in H)-sum(FS_bids['Quantity'][i]*FS_bids['price'][i] for i in range(L_FS))*sum(V_S[h,i]for i in range(L_FS) for h in H)
                +sum(Pair(1,h,HD_bids,'DQ')[0]-Pair(1,h,HD_bids,'DP')[0] for h in H)
                
,GRB.MAXIMIZE)

In [761]:
sum(Pair(j,2,HS_bids,'SP')[1]-Pair(j,2,HS_bids,'SP')[0] for j in J)

2000.0

In [762]:
sum(Pair(j,1,HD_bids,'DP')[1]-Pair(j,1,HD_bids,'DP')[0] for j in J)

-2000.000000000001

In [763]:
gamma = 100000000000000000000000000000000000

        # (2)
E.addConstrs(-sum((Pair(j,h,HD_bids,'DQ')[1]-Pair(j,h,HD_bids,'DQ')[0]) * xD[h,j] for j in J ) +sum((Pair(j,h,HS_bids,'SQ')[1]-Pair(j,h,HS_bids,'SQ')[0])*xS[h,j] for j in J) -Pair(1,h,HD_bids,'DQ')[0] +Pair(1,h,HS_bids,'SQ')[0] 
                    +sum(Delta_S[i,h-1]*BS_bids['Quantity'][i]*yS[i] for i in range(L_S))  -sum(Delta_D[i,h-1]*BD_bids['Quantity'][i]*yD[i] for i in range(L_D))
                    +sum(FS_bids['Quantity'][i]*V_S[h,i]for i in range(L_FS))-sum(FD_bids['Quantity'][i]*V_D[h,i]for i in range(L_FD))
                    
                    ==0 for h in H) 
 


E.addConstrs(F_z[h] == Sm_Res[h-1][0]+sum((Pair(j,h,HS_bids,'SP')[1]-Pair(j,h,HS_bids,'SP')[0]) * xS[h,j] for j in J)for h in H)        
E.addConstrs(F_z[h] == DMax_Res[h-1][0]+sum((Pair(j,h,HD_bids,'DP')[1]-Pair(j,h,HD_bids,'DP')[0]) * xD[h,j] for j in J) for h in H)
# (8)

# E.addConstrs(-BS_bids['Duration'][j]* BS_bids['price'][j]+ sum(Delta_S[j,h-1]*F_z[h] for h in H ) <= yS[j]*gamma for j in range(L_S))
# # (9)

# E.addConstrs(BD_bids['Duration'][j]*BD_bids['price'][j]- sum(Delta_S[j,h-1]*F_z[h] for h in H) <= yD[j]*gamma for j in range(L_D))

E.addConstrs(-BS_bids['Duration'][j]* BS_bids['price'][j]+ sum(Delta_S[j,h-1]*F_z[h] for h in H ) <= yS[j]*gamma for j in np.arange(0,L_S,1)[L_S_link_ID])
# (9)

E.addConstrs(BD_bids['Duration'][j]*BD_bids['price'][j]- sum(Delta_S[j,h-1]*F_z[h] for h in H) <= yD[j]*gamma for j in np.arange(0,L_D,1)[L_D_link_ID])
        
        
    # E.addConstr(if Blockbid['LinkID'][j] == 'NaN': Blockbid['Duration'][j]*Blockbid['price'][j]+ sum(Delta_S[j,h]*F_z[h] for j in J) <= yd[j]*gamma)
        
# (3)(4)
# Demand
E.addConstrs(w_D[h,1]<=xD[h,1] for h in H)
E.addConstrs(xD[h,1] <=1 for h in H)
E.addConstrs(w_D[h,j]<=xD[h,j] for j in range(2,find_largest(HIndex_D)) for h in H)
E.addConstrs(xD[h,j]<=w_D[h,j-1] for j in range(2,find_largest(HIndex_D)) for h in H)
E.addConstrs(0<= xD[h,999] for h in H)
E.addConstrs(xD[h,999] <= w_D[h,998] for h in H)

# Supply
E.addConstrs(w_S[h,1]<=xS[h,1] for h in H)
E.addConstrs(xS[h,1] <=1 for h in H)
E.addConstrs(w_S[h,j]<=xS[h,j] for j in range(2,find_largest(HIndex_S)) for h in H)
E.addConstrs(xS[h,j]<=w_S[h,j-1] for j in range(2,find_largest(HIndex_S)) for h in H)
E.addConstrs(0<= xS[h,999] for h in H)
E.addConstrs(xS[h,999] <= w_S[h,998] for h in H)


E.addConstrs(sum(V_S[h,i] for h in H )<= 1 for i in range(L_FS) )
E.addConstrs(sum(V_D[h,i] for h in H )<= 1 for i in range(L_FD) )

E.addConstrs(F_z[h]-FS_bids['price'][i] <= gamma*sum(V_S[k,i] for k in H) for i in range(L_FS) for h in H)
E.addConstrs(FD_bids['price'][i]-F_z[h]<=gamma*sum(V_D[k,i] for k in H) for i in range(L_FD) for h in H)




{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model Update*>,
 (0, 20): <gurobi.Constr *Awaiting Model

In [765]:
import time
Inital =time.time()
E.optimize()
Final =time.time()-Inital

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 97426 rows, 97328 columns and 324717 nonzeros
Model fingerprint: 0x0c73754e
Model has 47837 quadratic objective terms
Variable types: 47976 continuous, 49352 integer (49352 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+35]
  Objective range  [9e+00, 2e+07]
  QObjective range [2e-02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 2791 rows and 2246 columns
Presolve time: 2.08s
Presolved: 94635 rows, 95082 columns, 288101 nonzeros
Presolved model has 46718 quadratic objective terms
Variable types: 46857 continuous, 48225 integer (48225 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   18088    2.8061461e+08   0.000000e+0

In [772]:
E.getAttr('x',xD)

{(1, 1): 1.0,
 (1, 2): 1.0,
 (1, 3): 1.0,
 (1, 4): 1.0,
 (1, 5): 1.0,
 (1, 6): 1.0,
 (1, 7): 1.0,
 (1, 8): 1.0,
 (1, 9): 1.0,
 (1, 10): 1.0,
 (1, 11): 1.0,
 (1, 12): 1.0,
 (1, 13): 1.0,
 (1, 14): 1.0,
 (1, 15): 1.0,
 (1, 16): 1.0,
 (1, 17): 1.0,
 (1, 18): 1.0,
 (1, 19): 1.0,
 (1, 20): 1.0,
 (1, 21): 1.0,
 (1, 22): 1.0,
 (1, 23): 1.0,
 (1, 24): 1.0,
 (1, 25): 1.0,
 (1, 26): 1.0,
 (1, 27): 1.0,
 (1, 28): 1.0,
 (1, 29): 1.0,
 (1, 30): 1.0,
 (1, 31): 1.0,
 (1, 32): 1.0,
 (1, 33): 1.0,
 (1, 34): 1.0,
 (1, 35): 1.0,
 (1, 36): 1.0,
 (1, 37): 1.0,
 (1, 38): 1.0,
 (1, 39): 1.0,
 (1, 40): 1.0,
 (1, 41): 1.0,
 (1, 42): 1.0,
 (1, 43): 1.0,
 (1, 44): 1.0,
 (1, 45): 1.0,
 (1, 46): 1.0,
 (1, 47): 1.0,
 (1, 48): 1.0,
 (1, 49): 1.0,
 (1, 50): 1.0,
 (1, 51): 1.0,
 (1, 52): 1.0,
 (1, 53): 1.0,
 (1, 54): 1.0,
 (1, 55): 1.0,
 (1, 56): 1.0,
 (1, 57): 1.0,
 (1, 58): 1.0,
 (1, 59): 1.0,
 (1, 60): 1.0,
 (1, 61): 1.0,
 (1, 62): 1.0,
 (1, 63): 1.0,
 (1, 64): 1.0,
 (1, 65): 1.0,
 (1, 66): 1.0,
 (1, 67): 1.0,
 (1,

In [778]:
Index = [v.VarName for v in xS.values()]
value = [v.x for v in xS.values()]
Solution_HS = [[]for i in range(len(J))]
for j in range(len(J)):
    for i in range(24):
        Solution_HS[j].append(value[len(J)*i+j])

Solution_HS = pd.DataFrame(Solution_HS)
Solution_HS
Solution_HS.to_csv('Result/HS.csv',index=False,header=False)

In [779]:
Index = [v.VarName for v in xD.values()]
value = [v.x for v in xD.values()]
Solution_HD = [[]for i in range(len(J))]
for j in range(len(J)):
    for i in range(24):
        Solution_HD[j].append(value[len(J)*i+j])

Solution_HD = pd.DataFrame(Solution_HD)
Solution_HD.to_csv('Result/HD.csv',index=False,header=False)

In [780]:
def Find_H_Break(path,DF,SD):
    
    Data = pd.read_csv(path,header = None)
    Res=[[] for i in range(24)]
    for i in range(24):
        for j in range(len(Data)):
            if Data.values[j][i] >= 0.00001 and Data.values[j][i] <= 0.999999:
                Res[i].append(j)
                Res[i].append(Data.values[j][i])
                Res[i].append(sum((Pair(j,i+1,DF,SD)[1]-Pair(j,i+1,DF,SD)[0]) * Data.values[j-1][i] for j in range(1,len(Data)+1)))
                
    return Res

In [781]:
Find_H_Break(path='Result/HS.csv',DF=HS_bids,SD='SP')

[[46, 0.118473980607297, 92.05112461362593],
 [405, 0.3512622359608367, 810.2005924781041],
 [90, 0.9181275643416096, 180.5931555389779],
 [103, 0.4351739764719649, 205.86996368376916],
 [200, 0.3815984213122246, 399.706771090281],
 [384, 0.8522690698421608, 768.6594271000964],
 [46, 0.0746097726499847, 91.96146929080398],
 [62, 0.7056737588652469, 124.93978723404254],
 [155, 0.6990516977669068, 311.08260507800554],
 [409, 0.144877839691386, 818.5469995713673],
 [70, 0.7888258618790324, 141.10958365458742],
 [409, 0.3786937699981064, 818.3233596837945],
 [46, 0.3280238924837284, 92.05846689895444],
 [171, 0.2308506543495544, 341.30714780600476],
 [46, 0.5030536804885762, 91.90818386370938],
 [79, 0.9977163156559808, 159.82586653133734],
 [289, 0.3630214205185529, 578.6446110484779],
 [176, 0.712328767123267, 352.94205479452046],
 [332, 0.7946943483274911, 665.6892502883504],
 [133, 0.6567481402762665, 267.29200850159395],
 [130, 0.7731456988676516, 259.94855994074663],
 [46, 0.65603456

In [ ]:
2000+Find_H_Break(path='Result/HD.csv',DF=HD_bids,SD='DP')[0][2]


91.68355155482732

In [ ]:
Find_H_Break(path='Result/HD.csv',DF=HD_bids,SD='DP')

[[952, 0.5417348608837935, -1908.3164484451727],
 [593, 0.8826378155621861, -1189.7994075218937],
 [907, 0.9173815740393172, -1819.4068444610189],
 [896, 0.1048825154418434, -1794.1300363162295],
 [798, 0.5013566847545996, -1600.2932289097155],
 [614, 0.8709365947853149, -1231.3405728999023],
 [952, 0.9801444043318956, -1907.8776353790602],
 [936, 0.1453900709216314, -1875.0602127659572],
 [842, 0.9303762618535568, -1688.9173949219944],
 [589, 0.3180454350530946, -1181.4530004286291],
 [927, 0.7501239123249844, -1858.8904163454126],
 [589, 0.7336721249759218, -1181.6766403162028],
 [952, 0.5801670261598155, -1907.9534572202856],
 [828, 0.2925808314085789, -1658.692852193996],
 [952, 0.1370138219218752, -1907.8385470909686],
 [918, 0.5940962530655908, -1840.1741334686612],
 [709, 0.0372040586235851, -1421.3553889515194],
 [822, 0.1780821917798895, -1647.0579452054758],
 [666, 0.0198385236427526, -1334.310749711646],
 [864, 0.9893730074388896, -1732.7079914984088],
 [868, 0.6366522061157